In [ ]:
from math import pi

import matplotlib.pyplot as plt

In [ ]:
data = [
    ("V1", 8),
    ("V2", 10),
    ("V3", 9),
    ("V4", 12),
    ("V5", 6),
    ("V6", 14),
    ("V7", 15),
    ("V8", 25),
]

In [ ]:
def prepare_data(data):
    categories = tuple(d[0] for d in data)
    values = tuple(d[1] for d in data)

    return categories, values, len(categories)

In [ ]:
categories, values, N = prepare_data(data)